In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, kruskal
import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
class SolarDataAnalyzer:
    """Comprehensive solar data analysis for MoonLight Energy Solutions"""
    
    def __init__(self, filepath, country_name):
        """Initialize with data file path and country name"""
        self.country = country_name
        self.df = pd.read_csv(filepath)
        self.df_clean = None
        
    def initial_profiling(self):
        """Generate initial data profiling report"""
        print(f"{'='*60}")
        print(f"INITIAL DATA PROFILE - {self.country.upper()}")
        print(f"{'='*60}\n")
        
        # Basic info
        print(f"Dataset Shape: {self.df.shape}")
        print(f"Date Range: {self.df['Timestamp'].min()} to {self.df['Timestamp'].max()}\n")
        
        # Missing values report
        print("MISSING VALUES REPORT")
        print("-" * 60)
        missing = self.df.isna().sum()
        missing_pct = (missing / len(self.df)) * 100
        missing_df = pd.DataFrame({
            'Missing_Count': missing,
            'Percentage': missing_pct
        })
        missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Percentage', ascending=False)
        print(missing_df)
        
        # Columns with >5% nulls
        critical_missing = missing_df[missing_df['Percentage'] > 5]
        if len(critical_missing) > 0:
            print(f"\n⚠️  CRITICAL: {len(critical_missing)} columns have >5% missing values")
            print(critical_missing)
        else:
            print("\n✓ No columns have >5% missing values")
        
        # Summary statistics
        print(f"\n{'='*60}")
        print("SUMMARY STATISTICS")
        print(f"{'='*60}\n")
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns
        print(self.df[numeric_cols].describe())
        
        return missing_df
    
    def detect_outliers(self, columns=None):
        """Detect outliers using Z-score method (|Z| > 3)"""
        if columns is None:
            columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
        
        print(f"\n{'='*60}")
        print("OUTLIER DETECTION (Z-score method, |Z| > 3)")
        print(f"{'='*60}\n")
        
        outlier_summary = {}
        for col in columns:
            if col in self.df.columns:
                z_scores = np.abs(stats.zscore(self.df[col].dropna()))
                outliers = (z_scores > 3).sum()
                outlier_pct = (outliers / len(self.df)) * 100
                outlier_summary[col] = {
                    'count': outliers,
                    'percentage': outlier_pct
                }
                print(f"{col:15} : {outliers:5} outliers ({outlier_pct:.2f}%)")
        
        return outlier_summary
    
    def clean_data(self, strategy='median'):
        """Clean data by handling missing values and outliers"""
        print(f"\n{'='*60}")
        print("DATA CLEANING")
        print(f"{'='*60}\n")
        
        self.df_clean = self.df.copy()
        
        # Convert timestamp
        self.df_clean['Timestamp'] = pd.to_datetime(self.df_clean['Timestamp'])
        
        # Key columns for imputation
        key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'BP']
        
        # Impute missing values
        for col in key_columns:
            if col in self.df_clean.columns:
                missing_before = self.df_clean[col].isna().sum()
                if missing_before > 0:
                    if strategy == 'median':
                        self.df_clean[col].fillna(self.df_clean[col].median(), inplace=True)
                    elif strategy == 'mean':
                        self.df_clean[col].fillna(self.df_clean[col].mean(), inplace=True)
                    print(f"✓ Imputed {missing_before} missing values in {col}")
        
        # Handle negative values in irradiance (physically impossible)
        irradiance_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB']
        for col in irradiance_cols:
            if col in self.df_clean.columns:
                neg_count = (self.df_clean[col] < 0).sum()
                if neg_count > 0:
                    self.df_clean.loc[self.df_clean[col] < 0, col] = 0
                    print(f"✓ Corrected {neg_count} negative values in {col}")
        
        print(f"\n✓ Cleaned dataset shape: {self.df_clean.shape}")
        return self.df_clean
    
    def time_series_analysis(self):
        """Analyze time series patterns"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'Time Series Analysis - {self.country}', fontsize=16, fontweight='bold')
        
        # Extract time features
        self.df_clean['Hour'] = self.df_clean['Timestamp'].dt.hour
        self.df_clean['Month'] = self.df_clean['Timestamp'].dt.month
        self.df_clean['Date'] = self.df_clean['Timestamp'].dt.date
        
        # Daily pattern - GHI
        hourly_ghi = self.df_clean.groupby('Hour')['GHI'].mean()
        axes[0, 0].plot(hourly_ghi.index, hourly_ghi.values, marker='o', linewidth=2)
        axes[0, 0].set_title('Average GHI by Hour of Day', fontweight='bold')
        axes[0, 0].set_xlabel('Hour')
        axes[0, 0].set_ylabel('GHI (W/m²)')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Monthly pattern - GHI, DNI, DHI
        monthly_data = self.df_clean.groupby('Month')[['GHI', 'DNI', 'DHI']].mean()
        monthly_data.plot(ax=axes[0, 1], marker='o', linewidth=2)
        axes[0, 1].set_title('Monthly Average Irradiance', fontweight='bold')
        axes[0, 1].set_xlabel('Month')
        axes[0, 1].set_ylabel('Irradiance (W/m²)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Temperature vs Time
        hourly_temp = self.df_clean.groupby('Hour')['Tamb'].mean()
        axes[1, 0].plot(hourly_temp.index, hourly_temp.values, marker='o', 
                       color='orangered', linewidth=2)
        axes[1, 0].set_title('Average Temperature by Hour', fontweight='bold')
        axes[1, 0].set_xlabel('Hour')
        axes[1, 0].set_ylabel('Temperature (°C)')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Daily trends
        daily_ghi = self.df_clean.groupby('Date')['GHI'].mean()
        axes[1, 1].plot(daily_ghi.index, daily_ghi.values, linewidth=1, alpha=0.7)
        axes[1, 1].set_title('Daily Average GHI Trend', fontweight='bold')
        axes[1, 1].set_xlabel('Date')
        axes[1, 1].set_ylabel('GHI (W/m²)')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    def cleaning_impact_analysis(self):
        """Analyze impact of cleaning on module performance"""
        if self.df_clean is None or 'Cleaning' not in self.df_clean.columns:
            print("⚠️  Cleaning data not available")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle(f'Cleaning Impact Analysis - {self.country}', fontsize=16, fontweight='bold')
        
        cleaning_impact = self.df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()
        
        # ModA
        cleaning_impact['ModA'].plot(kind='bar', ax=axes[0], color=['coral', 'lightblue'])
        axes[0].set_title('ModA Performance: Pre vs Post Cleaning', fontweight='bold')
        axes[0].set_xlabel('Cleaning Status (0=Before, 1=After)')
        axes[0].set_ylabel('Average ModA (W/m²)')
        axes[0].set_xticklabels(['Before Cleaning', 'After Cleaning'], rotation=0)
        
        # ModB
        cleaning_impact['ModB'].plot(kind='bar', ax=axes[1], color=['coral', 'lightblue'])
        axes[1].set_title('ModB Performance: Pre vs Post Cleaning', fontweight='bold')
        axes[1].set_xlabel('Cleaning Status (0=Before, 1=After)')
        axes[1].set_ylabel('Average ModB (W/m²)')
        axes[1].set_xticklabels(['Before Cleaning', 'After Cleaning'], rotation=0)
        
        plt.tight_layout()
        plt.show()
        
        # Print statistics
        print("\nCleaning Impact Statistics:")
        print(cleaning_impact)
        
        if len(cleaning_impact) > 1:
            improvement_a = ((cleaning_impact.loc[1, 'ModA'] - cleaning_impact.loc[0, 'ModA']) 
                           / cleaning_impact.loc[0, 'ModA'] * 100)
            improvement_b = ((cleaning_impact.loc[1, 'ModB'] - cleaning_impact.loc[0, 'ModB']) 
                           / cleaning_impact.loc[0, 'ModB'] * 100)
            print(f"\n✓ ModA improvement after cleaning: {improvement_a:.2f}%")
            print(f"✓ ModB improvement after cleaning: {improvement_b:.2f}%")
    
    def correlation_analysis(self):
        """Analyze correlations between variables"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        # Select relevant columns
        corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS', 'BP']
        corr_cols = [col for col in corr_cols if col in self.df_clean.columns]
        
        # Correlation heatmap
        plt.figure(figsize=(12, 8))
        correlation_matrix = self.df_clean[corr_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                   center=0, square=True, linewidths=1)
        plt.title(f'Correlation Matrix - {self.country}', fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        # Scatter plots
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle(f'Relationship Analysis - {self.country}', fontsize=16, fontweight='bold')
        
        # WS vs GHI
        axes[0, 0].scatter(self.df_clean['WS'], self.df_clean['GHI'], alpha=0.3, s=10)
        axes[0, 0].set_xlabel('Wind Speed (m/s)')
        axes[0, 0].set_ylabel('GHI (W/m²)')
        axes[0, 0].set_title('Wind Speed vs GHI')
        axes[0, 0].grid(True, alpha=0.3)
        
        # RH vs Tamb
        axes[0, 1].scatter(self.df_clean['RH'], self.df_clean['Tamb'], alpha=0.3, s=10, color='green')
        axes[0, 1].set_xlabel('Relative Humidity (%)')
        axes[0, 1].set_ylabel('Temperature (°C)')
        axes[0, 1].set_title('Relative Humidity vs Temperature')
        axes[0, 1].grid(True, alpha=0.3)
        
        # RH vs GHI
        axes[1, 0].scatter(self.df_clean['RH'], self.df_clean['GHI'], alpha=0.3, s=10, color='orange')
        axes[1, 0].set_xlabel('Relative Humidity (%)')
        axes[1, 0].set_ylabel('GHI (W/m²)')
        axes[1, 0].set_title('Relative Humidity vs GHI')
        axes[1, 0].grid(True, alpha=0.3)
        
        # DNI vs DHI
        axes[1, 1].scatter(self.df_clean['DNI'], self.df_clean['DHI'], alpha=0.3, s=10, color='purple')
        axes[1, 1].set_xlabel('DNI (W/m²)')
        axes[1, 1].set_ylabel('DHI (W/m²)')
        axes[1, 1].set_title('Direct vs Diffuse Irradiance')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def wind_analysis(self):
        """Analyze wind patterns"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle(f'Wind Analysis - {self.country}', fontsize=16, fontweight='bold')
        
        # Wind speed histogram
        axes[0].hist(self.df_clean['WS'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
        axes[0].set_xlabel('Wind Speed (m/s)')
        axes[0].set_ylabel('Frequency')
        axes[0].set_title('Wind Speed Distribution')
        axes[0].axvline(self.df_clean['WS'].mean(), color='red', 
                       linestyle='--', label=f'Mean: {self.df_clean["WS"].mean():.2f} m/s')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Wind direction polar plot (simplified)
        if 'WD' in self.df_clean.columns:
            # Bin wind directions
            wind_bins = np.arange(0, 361, 30)
            wd_binned = pd.cut(self.df_clean['WD'], bins=wind_bins)
            wd_counts = wd_binned.value_counts().sort_index()
            
            angles = np.arange(15, 361, 30) * np.pi / 180
            axes[1].bar(angles, wd_counts.values, width=0.5, alpha=0.7, color='lightcoral')
            axes[1].set_theta_zero_location('N')
            axes[1].set_theta_direction(-1)
            axes[1].set_title('Wind Direction Distribution')
        
        plt.tight_layout()
        plt.show()
    
    def distribution_analysis(self):
        """Analyze distributions of key variables"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle(f'Distribution Analysis - {self.country}', fontsize=16, fontweight='bold')
        
        # GHI histogram
        axes[0, 0].hist(self.df_clean['GHI'], bins=50, edgecolor='black', alpha=0.7, color='gold')
        axes[0, 0].set_xlabel('GHI (W/m²)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].set_title('GHI Distribution')
        axes[0, 0].axvline(self.df_clean['GHI'].mean(), color='red', 
                          linestyle='--', label=f'Mean: {self.df_clean["GHI"].mean():.2f}')
        axes[0, 0].legend()
        
        # DNI histogram
        axes[0, 1].hist(self.df_clean['DNI'], bins=50, edgecolor='black', alpha=0.7, color='orange')
        axes[0, 1].set_xlabel('DNI (W/m²)')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].set_title('DNI Distribution')
        axes[0, 1].axvline(self.df_clean['DNI'].mean(), color='red', 
                          linestyle='--', label=f'Mean: {self.df_clean["DNI"].mean():.2f}')
        axes[0, 1].legend()
        
        # Temperature histogram
        axes[1, 0].hist(self.df_clean['Tamb'], bins=50, edgecolor='black', alpha=0.7, color='coral')
        axes[1, 0].set_xlabel('Temperature (°C)')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].set_title('Temperature Distribution')
        axes[1, 0].axvline(self.df_clean['Tamb'].mean(), color='red', 
                          linestyle='--', label=f'Mean: {self.df_clean["Tamb"].mean():.2f}')
        axes[1, 0].legend()
        
        # Relative Humidity histogram
        axes[1, 1].hist(self.df_clean['RH'], bins=50, edgecolor='black', alpha=0.7, color='lightblue')
        axes[1, 1].set_xlabel('Relative Humidity (%)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Relative Humidity Distribution')
        axes[1, 1].axvline(self.df_clean['RH'].mean(), color='red', 
                          linestyle='--', label=f'Mean: {self.df_clean["RH"].mean():.2f}')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()
    
    def temperature_humidity_analysis(self):
        """Analyze relationship between temperature, humidity and solar radiation"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle(f'Temperature & Humidity Analysis - {self.country}', 
                    fontsize=16, fontweight='bold')
        
        # Create humidity bins
        self.df_clean['RH_bin'] = pd.cut(self.df_clean['RH'], bins=5)
        
        # Temperature by humidity bins
        rh_temp = self.df_clean.groupby('RH_bin')['Tamb'].mean()
        rh_temp.plot(kind='bar', ax=axes[0], color='teal', alpha=0.7)
        axes[0].set_title('Average Temperature by Humidity Level')
        axes[0].set_xlabel('Relative Humidity Range (%)')
        axes[0].set_ylabel('Temperature (°C)')
        axes[0].tick_params(axis='x', rotation=45)
        
        # GHI by humidity bins
        rh_ghi = self.df_clean.groupby('RH_bin')['GHI'].mean()
        rh_ghi.plot(kind='bar', ax=axes[1], color='orange', alpha=0.7)
        axes[1].set_title('Average GHI by Humidity Level')
        axes[1].set_xlabel('Relative Humidity Range (%)')
        axes[1].set_ylabel('GHI (W/m²)')
        axes[1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    def bubble_chart(self):
        """Create bubble chart: GHI vs Tamb with RH as bubble size"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        # Sample data for better visualization
        sample_data = self.df_clean.sample(min(1000, len(self.df_clean)))
        
        plt.figure(figsize=(12, 7))
        scatter = plt.scatter(sample_data['Tamb'], sample_data['GHI'], 
                            s=sample_data['RH']*2, alpha=0.5, 
                            c=sample_data['RH'], cmap='viridis', edgecolors='black', linewidth=0.5)
        
        plt.colorbar(scatter, label='Relative Humidity (%)')
        plt.xlabel('Ambient Temperature (°C)', fontsize=12)
        plt.ylabel('GHI (W/m²)', fontsize=12)
        plt.title(f'GHI vs Temperature (Bubble Size = Relative Humidity) - {self.country}', 
                 fontsize=14, fontweight='bold', pad=20)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def export_cleaned_data(self, output_path):
        """Export cleaned data to CSV"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        self.df_clean.to_csv(output_path, index=False)
        print(f"✓ Cleaned data exported to: {output_path}")
    
    def generate_insights_report(self):
        """Generate key insights for strategic recommendations"""
        if self.df_clean is None:
            print("⚠️  Please run clean_data() first")
            return
        
        print(f"\n{'='*70}")
        print(f"KEY INSIGHTS REPORT - {self.country.upper()}")
        print(f"{'='*70}\n")
        
        # Solar potential metrics
        print("1. SOLAR POTENTIAL METRICS")
        print("-" * 70)
        print(f"   Average GHI: {self.df_clean['GHI'].mean():.2f} W/m²")
        print(f"   Peak GHI: {self.df_clean['GHI'].max():.2f} W/m²")
        print(f"   Average DNI: {self.df_clean['DNI'].mean():.2f} W/m²")
        print(f"   Average DHI: {self.df_clean['DHI'].mean():.2f} W/m²")
        print(f"   GHI Variability (Std Dev): {self.df_clean['GHI'].std():.2f} W/m²")
        
        # Environmental conditions
        print(f"\n2. ENVIRONMENTAL CONDITIONS")
        print("-" * 70)
        print(f"   Average Temperature: {self.df_clean['Tamb'].mean():.2f} °C")
        print(f"   Average Humidity: {self.df_clean['RH'].mean():.2f} %")
        print(f"   Average Wind Speed: {self.df_clean['WS'].mean():.2f} m/s")
        print(f"   Average Pressure: {self.df_clean['BP'].mean():.2f} hPa")
        
        # Module performance
        if 'ModA' in self.df_clean.columns and 'ModB' in self.df_clean.columns:
            print(f"\n3. MODULE PERFORMANCE")
            print("-" * 70)
            print(f"   Average ModA: {self.df_clean['ModA'].mean():.2f} W/m²")
            print(f"   Average ModB: {self.df_clean['ModB'].mean():.2f} W/m²")
            
            # Module temperature impact
            if 'TModA' in self.df_clean.columns:
                print(f"   Average Module Temp A: {self.df_clean['TModA'].mean():.2f} °C")
            if 'TModB' in self.df_clean.columns:
                print(f"   Average Module Temp B: {self.df_clean['TModB'].mean():.2f} °C")
        
        # Temporal patterns
        print(f"\n4. TEMPORAL PATTERNS")
        print("-" * 70)
        peak_hour = self.df_clean.groupby('Hour')['GHI'].mean().idxmax()
        peak_month = self.df_clean.groupby('Month')['GHI'].mean().idxmax()
        print(f"   Peak Solar Hour: {peak_hour}:00")
        print(f"   Peak Solar Month: {peak_month}")
        
        # Data quality
        print(f"\n5. DATA QUALITY SCORE")
        print("-" * 70)
        completeness = (1 - self.df.isna().sum().sum() / (self.df.shape[0] * self.df.shape[1])) * 100
        print(f"   Data Completeness: {completeness:.2f}%")
        
        print(f"\n{'='*70}\n")